## The required format for the GLAD program is:
```
First line:
    <numGivenLabels> <numLabelers> <numTasks> <Prior p(Z=1)>
  Following <numGivenLabels> lines:
    <taskId> <labelerId> <label:0|1>
```
0: Not a Disease (Includes "Phenotype")
<br>
1: Disease<br>
2: Phenotype<br>
3: Both

I think task refers to the word in the dataset, and then labeler ID and the label. So if we have 20000 words in M2C and 5 label, the output of this program should have 100000 lines.

### TODO: Disease and Phenotype M2C datasets do not have "both" label in them. Pre process them to allow for both!

In [36]:
dataset_folder = 'dataset/Mark2Cure_NER_dataset/'

In [37]:
citizens = ['1', '2', '3', '4', '5']

In [38]:
filename = 'M2C{}-Disease_Phen_Preliminary_Run_1st_pass.txt'

In [39]:
text = ''

In [74]:
## Make vocab
vocab = {}
reverse_vocab = {}
i = 0
total_lines = 0

for citizen in citizens:
    path = dataset_folder + filename.format(citizen)
    with open(path) as f:
        lines = f.read().split('\n')
        for line in lines:
            if line and line.split()[3] != 'O':
                total_lines += 1
                word = line.split()[0]
                if word.lower() not in vocab:
                    vocab[word.lower()] = i
                    reverse_vocab[i] = word.lower()
                    i += 1

In [75]:
len(vocab)

1279

In [76]:
# make 5 arrays of text
annotations = []
for citizen in citizens:
    path = dataset_folder + filename.format(citizen)
    with open(path) as f:
        lines = f.read().split('\n')
        annotations.append(lines)

In [77]:
text = '{} {} {} {} {} {} {}\n'.format(len(vocab)*len(annotations), len(annotations), len(vocab), 3, 0.33, 0.33, 0.34)

In [78]:
len(annotations), total_lines

(5, 11894)

In [80]:
record = dict()
task_arr = []
some_counter = 0
text = ''
for line in range(total_lines):
    for annotator in range(len(annotations)):
        try:
            current_line = annotations[annotator][line].split()
        except:
            continue
        if len(current_line) != 4:
            continue
        word = current_line[0].lower()
        if word == 1294:
            continue
        annotation = current_line[3]
        count = record.get(word, 0)
        if count < len(annotations):
            if annotation == "O":
                continue
            elif annotation == "I-Disease":
                label = '0'
            elif annotation == "I-Phenotype":
                label = '1'
            else:
                label = '2'
            some_counter += 1
            if word not in task_arr:
                task_arr.append(word)
            text = text + '{} {} {}\n'.format(vocab[word], annotator, label)
            record[word] = count + 1

In [81]:
text = '{} {} {} {} {} {} {}\n'.format(some_counter, len(annotations), len(task_arr), 3, 0.33, 0.33, 0.34) + text
with open('data_combined.txt', 'w') as f:
    f.write(text)

## The following code is to convert the alpha, beta and label csv to readable formats AFTER we run GLAD

### Make sure you have the proper files in the proper folder before running these hacky snippets

In [82]:
results_folder = 'Results For Combined Without O/'
files = ['beta.csv', 'label.csv']

In [87]:
text = 'Word,Beta\n'
with open(results_folder+'beta.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            text += '{},{}'.format(reverse_vocab[int(num)-1], arr[1])
with open(results_folder+'beta_new_combined_without_O.csv','w') as g:
    g.write(text)

In [88]:
text = 'Word,Disease,Phenotype,Both\n'
with open(results_folder+'label.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            word = reverse_vocab[int(num)-1]
            if num == '1294':
                continue
            text += '{},{},{},{}'.format(reverse_vocab[int(num)-1], arr[1], arr[2], arr[3])
with open(results_folder+'label_new_combined_without_O.csv','w') as g:
    g.write(text)

In [51]:
vocab['1294']

KeyError: '1294'